In [2]:
from os import listdir
from os.path import join as pjoin
from os.path import basename, isfile

import numpy as np
import pandas as pd
from scipy.io import loadmat

# Data conversion

Converting .mat files into dataframes and then csv files

In [1]:
def convert_to_csv(file):
    mat = loadmat(file)
    df = pd.DataFrame()
    df['leftP'] = mat['leftP'][0]
    df['rightP'] = mat['rightP'][0]
    df['trial_response_side'] = mat['trialresponseside'][0]
    df['trial_reward'] = mat['trialreward'][0]
    return df

def convert_timing_to_csv(df, file):
    mat = loadmat(file)
    df['cue_time'] = mat['cuetimes'][0]
    # fill this column with nan
    df['reward_time'] = np.nan
    # assign rewarded non-nan trials' response time 
    # using the rewardtimes array from the mat file
    df.loc[df['trial_reward'] == 1, 'reward_time'] = mat['rewardtimes'][0]
    return df

In [3]:
# load .mat file
for file in listdir(pjoin('behaviour_data', 'mat', 'task_info')):
    path = pjoin('behaviour_data', 'mat', 'task_info', file)
    df = convert_to_csv(path) 
    timing_path = pjoin('behaviour_data', 'mat', 'timing_info', file)
    df = convert_timing_to_csv(df, timing_path)
    df.to_csv(pjoin('behaviour_data', file.split('.')[0] + '.csv'), index=False)

## convert spike times

In [20]:
from os import listdir
from os.path import join as pjoin, isfile, basename

from scipy.io import loadmat # type: ignore
import numpy as np

spike_data_root = pjoin('spike_times', 'sessions')

for dir in listdir(spike_data_root):
    file = pjoin(spike_data_root, dir, 'spts.mat')
    if not isfile(file):
        print(dir)
        continue
    mat = loadmat(file)
    pfc = [np.array(i).flatten() for i in mat['pfcspts'][0]]
    dms = [np.array(i).flatten() for i in mat['strspts'][0]]
    pfc_np = np.empty(len(pfc), dtype=object)
    pfc_np[:] = pfc
    dms_np = np.empty(len(dms), dtype=object)
    dms_np[:] = dms
    np.save(pjoin(spike_data_root, dir, 'pfc.npy'), pfc_np)
    np.save(pjoin(spike_data_root, dir, 'dms.npy'), dms_np)

    for i in range(len(pfc)):
        np.save(pjoin(spike_data_root, dir, 'pfc_{}.npy'.format(i)), pfc[i])
    
    for i in range(len(dms)):
        np.save(pjoin(spike_data_root, dir, 'dms_{}.npy'.format(i)), dms[i])

# Convert Wei's summary table



In [ ]:
from os.path import join as pjoin, isdir, basename, isfile
from os import listdir, mkdir
import numpy as np
from glob import glob
from tqdm import tqdm 
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

data = pd.read_excel('PPD_RTW_summary_table_Wei_06may2022_V2.xlsx')

# rename the sessions to match the current format
# from index_yy.mm.dd to indexyyyymmdd
data['session'] = data['session'].apply(lambda x: x.replace('.', ''))
data['session'] = data['session'].apply(lambda x: x.replace('_', '20'))

# lower case the cell location + (cell ID -1)
data['cell'] = data['Cell location'].apply(lambda x: x.lower()) + '_' + (data['Cell ID'] - 1).astype(dms)
# remove the cell location and cell ID columns
data = data.drop(columns=['Cell location', 'Cell ID'])

# create another csv file storing stimulus, movement and reward correlation information
stimulus_movement_reward = data[['session','Stimulus_related_firing_P_value','Movement_related_firing_P_value','Reward_related_firing_P_value']]
# drop these columns from the original data
data = data.drop(columns=['Stimulus_related_firing_P_value','Movement_related_firing_P_value','Reward_related_firing_P_value'])
# rename the columns
data = data.rename(columns={'Background_firing_correlation_Coefficient_with_PPD': 'background_firing_pearson_r', 'Background_firing_correlation_P_ value_with PPD': 'background_firing_p_values', 'Response_magnitude_correlation_Coefficient_with_PPD': 'response_firing_pearson_r', 'Response_magnitude_correlation_P_ value_with PPD':'response_firing_p_values'})


# save the data in csv format
data.to_csv('delta_P_correlation.csv', index=False)

# save the stimulus, movement and reward correlation information in csv format
stimulus_movement_reward.to_csv('stimulus_movement_reward_correlation.csv', index=False)